<h1 align=center><b>Fine-Tuning Pre-Trained Language Models for Dialogue Summarization Using PEFT and LoRA</b></font></h1>

<br>

<p  align="center">
<img src="https://raw.githubusercontent.com/doguilmak/FineTune-DiaSum-PEFT-LoRA/main/assets/pexels-googledeepmind-18069696.jpg" height=500 width=2000 alt="Cover">
</p>  

<small>Picture Source: <a href="https://www.pexels.com/@googledeepmind/">Google Deepmind Pexels</a></small>

<br>

<h2>Abstract</h2>

In this notebook, we will explore the use of Parameter-Efficient Fine-Tuning (PEFT) to fine-tune a pre-trained sequence-to-sequence language model for dialogue summarization. PEFT allows us to adapt pre-trained models to new tasks efficiently, using fewer trainable parameters and less computational resources compared to traditional fine-tuning methods. This approach is particularly useful when dealing with large models and limited computational resources.

The main steps involved in this process are:

<br>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<li><a href="">Loading the Dataset:</a> We'll use the dialogsum dataset from Hugging Face's dataset library.</li>
<li><a href="">Preparing the Model</a>: We'll load the flan-t5-base model and its tokenizer.</li>
<li><a href="">Training the Model</a>: We'll configure PEFT and use it to fine-tune the model on our dataset.</li>
<li><a href="">Evaluation</a>: We'll generate summaries using the fine-tuned model and compare them to the baseline summaries provided in the dataset.</li>

</div>

<br>

Make sure your runtime is **GPU** (_not_ CPU or TPU). And if it is an option, make sure you are using _Python 3_. You can select these settings by going to `Runtime -> Change runtime type -> Select the above mentioned settings and then press SAVE`.

## **0. Install Necessary Libraries**

Before starting, we need to install the required libraries for the project, including `torch` and `datasets`.

In [ ]:
# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# %pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0 \
#     evaluate==0.4.0 \
#     rouge_score==0.1.2 \
#     loralib==0.1.1 \
#     peft==0.3.0 --quiet

## **1. Loading the Dataset**

We import the necessary modules for handling datasets, working with the pre-trained model, configuring training arguments, and performing evaluations.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"  # @param {type: "string"}

In [ ]:
from datasets import load_dataset

dataset = load_dataset(huggingface_dataset_name)
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

## **2. Preparing the Model**

We load the dialogsum dataset from Hugging Face's dataset library. This dataset contains dialogue summaries which we will use to fine-tune our model.

In [ ]:
model_name = 'google/flan-t5-base'  # @param {type: "string"}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

(…)le/flan-t5-base/resolve/main/config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)base/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## **3. Training the Model**

Here, we set up the PEFT configuration and train the model. We use **LoRA (Low-Rank Adaptation)** to fine-tune the model efficiently, allowing us to adapt it to our specific task with fewer trainable parameters. Before delve in, let's briely get into PEFT and LoRA:

<br>

### **PEFT (Parameter-Efficient Fine-Tuning)**

PEFT (Parameter-Efficient Fine-Tuning) is a technique designed to fine-tune pre-trained models on specific tasks using fewer parameters and computational resources compared to traditional fine-tuning methods. The key idea behind PEFT is to modify only a small subset of the model parameters, keeping the majority of the pre-trained model fixed. This approach reduces the memory and computational requirements, making it feasible to adapt large models to new tasks with limited resources.

#### Key Benefits of PEFT:
1. **Reduced Computational Resources**: By fine-tuning only a small portion of the model parameters, PEFT requires less memory and computational power.
2. **Faster Training**: Since fewer parameters are being updated, the training process is faster compared to full fine-tuning.
3. **Effective for Large Models**: PEFT is particularly useful for adapting large pre-trained models to new tasks without the need for extensive computational infrastructure.

<br>

### **LoRA (Low-Rank Adaptation)**

LoRA (Low-Rank Adaptation) is a specific implementation of PEFT that introduces a low-rank decomposition to the model parameters being fine-tuned. Instead of updating the full weight matrices of the model, LoRA updates a low-rank approximation of these matrices. This technique significantly reduces the number of trainable parameters, making the fine-tuning process more efficient.

#### How LoRA Works:
1. **Low-Rank Decomposition**: LoRA decomposes the weight matrices into two smaller matrices with a lower rank. This reduces the number of parameters that need to be updated during fine-tuning.
2. **Injecting Low-Rank Updates**: During training, only the low-rank matrices are updated, while the original pre-trained weights remain fixed.
3. **Efficient Adaptation**: By updating only a low-rank approximation, LoRA allows the model to adapt to new tasks efficiently without compromising on performance.

#### Key Benefits of LoRA:
1. **Parameter Efficiency**: LoRA significantly reduces the number of trainable parameters, making fine-tuning more memory-efficient.
2. **Scalability**: LoRA can be applied to very large models, enabling their adaptation to new tasks with limited resources.
3. **Maintaining Performance**: Despite the reduced number of trainable parameters, LoRA maintains competitive performance compared to traditional fine-tuning methods.

### **3.1. PEFT Configuration**

We configure the PEFT settings, specifying parameters such as the rank (r), alpha, dropout rate, and task type.

In [ ]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    r=8, # Rank of the low-rank decomposition. Controls the size of low-rank matrices.
    lora_alpha=32, # Scaling factor for the low-rank matrices. Balances the contribution of the low-rank adaptation.
    lora_dropout=0.05, # Dropout rate applied to the low-rank adaptation. Prevents overfitting by randomly dropping some adaptations.
    bias="none", # Type of bias adjustment. "none" indicates no bias terms are used in the low-rank adaptation.
    task_type=TaskType.SEQ_2_SEQ_LM # Task type for the model. Here, it's set for sequence-to-sequence language modeling.
)

peft_model = get_peft_model(original_model, lora_config)

### **3.2. Print Model Parameters**

We build `print_trainable_parameters(model)` function calculates and prints the number of trainable parameters in the model, providing insights into the model's size and complexity.

In [ ]:
def print_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params}")
    print(f"Total parameters: {total_params}")
    print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")

print_trainable_parameters(peft_model)

### **3.3. Training Arguments and Trainer**



We define the training arguments and initialize the trainer. The trainer is then used to fine-tune the model using our dataset.

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)

trainer.train()

### **3.4. Saving the Fine-Tuned Model**

After training, we save the fine-tuned model and tokenizer to a specified directory for later use.

In [ ]:
peft_model.save_pretrained("./peft_model")
tokenizer.save_pretrained("./peft_model")

## **4. Evaluation**

We load the fine-tuned model and tokenizer to evaluate its performance. We generate summaries for sample dialogues and compare them with the baseline summaries provided in the dataset.

### **4.1. Loading the Fine-Tuned Model**

We load the fine-tuned model and tokenizer for evaluation purposes.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./peft_model")
tokenizer = AutoTokenizer.from_pretrained("./peft_model")

### **4.2. Generating and Printing Summaries**

We use the fine-tuned model to generate summaries for a sample dialogue and print the results for comparison.

In [ ]:
def generate_summary(index):
    dialogue = dataset['test'][index]['dialogue']
    baseline_summary = dataset['test'][index]['summary']

    prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary:
    """

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = fine_tuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
        num_beams=4,
        early_stopping=True
    )
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Dialogue: {dialogue}")
    print(f"Baseline Summary: {baseline_summary}")
    print(f"Generated Summary: {generated_summary}")

In [ ]:
generate_summary(200)

<i>This structured approach ensures that we efficiently fine-tune the pre-trained model using PEFT and evaluate its performance on the dialogue summarization task.</i>

<br>

---

